In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit
using BenchmarkTools

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [11]:
Nx = 5
Ne = 300
X = randn(Nx, Ne).^2
Xvalid = randn(Nx, Ne).^2
λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [13]:
@time Cgreedy = greedyfit(Nx, 2, 2, deepcopy(X), 3, λ, δ, γ)

  0.003052 seconds (52.05 k allocations: 7.718 MiB)


Sparse radial map component of dimension 5 and order p = [2, 2, 2, -1, 2]


In [20]:
@time Cgreedy = greedyfit(Nx, 2, 1, deepcopy(X), deepcopy(Xvalid), 4, λ, δ, γ; verbose = false)

  0.006211 seconds (74.36 k allocations: 7.912 MiB)


Sparse radial map component of dimension 5 and order p = [2, 2, 2, 2, 1]


In [57]:
C = SparseRadialMapComponent(Nx, Cgreedy.p)

5-th component of a Sparse KR rearrangement of order p = [2, 2, -1, 2, 2]


In [58]:
@time center_std!(C, sort(deepcopy(X); dims = 2); γ = γ)

  0.000122 seconds (44 allocations: 62.719 KiB)


In [60]:
@time x_opt = optimize(C, X, λ, δ)
modify_a!(C, x_opt)

  0.002689 seconds (18.33 k allocations: 2.522 MiB)


In [65]:
negative_likelihood(C, X)

0.4260154881199424

In [66]:
ncoeff(C)

LoadError: [91mMethodError: no method matching ncoeff(::SparseRadialMapComponent)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  ncoeff([91m::LinHermiteMapComponent[39m) at /media/mat/HDD/AdaptiveTransportMap/src/hermitemap/linhermitemapcomponent.jl:32[39m
[91m[0m  ncoeff([91m::HermiteMapComponent[39m) at /media/mat/HDD/AdaptiveTransportMap/src/hermitemap/hermitemapcomponent.jl:85[39m
[91m[0m  ncoeff([91m::Int64[39m, [91m::Array{Int64,N} where N[39m) at /media/mat/HDD/AdaptiveTransportMap/src/radialmap/weights.jl:349[39m